In [1]:
import pandas as pd

In [2]:
def query(text):
    config = 'config.json'
    from model import query
    out = query(text, config)

    return out

In [3]:
test_sub = pd.read_csv('test/submit_2.csv')

In [4]:
out = []
for i in test_sub['вопрос']:
    out.append(query(i))

[Document(page_content='Кто должен регистрироваться в системе?   В Системе регистрируются товаропроизводители, которые проводят операции с зерном и продуктами  переработки зерна.   Перечень зерна (ссылка)'), Document(page_content='При перевозке разным видами транспорта, как оформить СДИЗ на перевозку?   При осуществлении перевозки разными видами транспорта требуется внести в раздел «Перевозка»  все виды и номера ТС, которые будут использованы во время одной перевозке. В случае перевалки  необходимо указать место перевалки.'), Document(page_content='путём формирования партий зерна. Если после проведения хранения, очистки, сушки и прочих  технологических процессов, операции с зерном не проводятся (отгрузка / приё мка, реализация,  перевозка), то вносить изменения в части веса или качества в сформированную ранее партию не  нужно. А при проведении отгрузки / приёмки, реализации или перевозки, необходимо создать новую  партии зерна, в которой указывается фактические вес и качество новой пар

In [5]:
data = pd.DataFrame(test_sub)
data['Ответ'] = out
data.to_csv('test/sub_2.csv')